<a href="https://colab.research.google.com/github/oxygen0605/ImageClassification/blob/master/deep_cnn_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colaboratory環境の初期設定

## Google Driveにマウントしてマシンスペックを出力

In [0]:
from google.colab import drive 
drive.mount('/content/drive')

!nvidia-smi > '/content/drive/My Drive/Colab Notebooks/Logs/machine_spec.txt'
!cat /proc/driver/nvidia/gpus/0000:00:04.0/information >> '/content/drive/My Drive/Colab Notebooks/Logs/machine_spec.txt'
!cat /etc/issue >> '/content/drive/My Drive/Colab Notebooks/Logs/machine_spec.txt'
!cat /proc/cpuinfo >> '/content/drive/My Drive/Colab Notebooks/Logs/machine_spec.txt'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Deep CNN (CIFAR-10)

## モデルの生成

In [0]:
from keras.models import Sequential, Model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Dense, BatchNormalization
from keras.layers import Input
from keras.layers.core import Activation, Flatten
from keras.applications.vgg16 import VGG16
from keras import regularizers

def deep_cnn(input_shape, num_classes):
    inputs = Input(shape = input_shape)
    
    x = Conv2D(64,(3,3),padding = "SAME",activation= "relu")(inputs)
    x = Conv2D(64,(3,3),padding = "SAME",activation= "relu")(x)
    x = BatchNormalization()(x)
    x = Conv2D(64,(3,3),padding = "SAME",activation= "relu")(x)
    x = MaxPooling2D()(x)
    x = Dropout(0.25)(x)

    x = Conv2D(128,(3,3),padding = "SAME",activation= "relu")(x)
    x = Conv2D(128,(3,3),padding = "SAME",activation= "relu")(x)
    x = BatchNormalization()(x)
    x = Conv2D(128,(3,3),padding = "SAME",activation= "relu")(x)
    x = MaxPooling2D()(x)
    x = Dropout(0.25)(x)

    x = Conv2D(256,(3,3),padding = "SAME",activation= "relu")(x)
    x = Conv2D(256,(3,3),padding = "SAME",activation= "relu")(x)
    x = BatchNormalization()(x)
    x = Conv2D(256,(3,3),padding = "SAME",activation= "relu")(x)
    x = Conv2D(256,(3,3),padding = "SAME",activation= "relu")(x)
    x = Conv2D(256,(3,3),padding = "SAME",activation= "relu")(x)
    x = BatchNormalization()(x)
    x = Conv2D(512,(3,3),padding = "SAME",activation= "relu")(x)
    x = Conv2D(512,(3,3),padding = "SAME",activation= "relu")(x)
    x = GlobalAveragePooling2D()(x)

    x = Dense(1024,activation = "relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(1024,activation = "relu")(x)
    x = Dropout(0.5)(x)
    y  = Dense(num_classes, activation = "softmax")(x)

    return Model(input = inputs, output = y)

Using TensorFlow backend.


## CIFAR10 データセットの用意

In [0]:
import keras
from keras.datasets import cifar10

class CIFAR10Dataset():
	def __init__(self):
		self.image_shape = (32, 32, 3)
		self.num_classes = 10
		
	def preprocess(self, data, label_data=False):
		if label_data:
			# conver class number to one-hot vector
			data = keras.utils.to_categorical(data, self.num_classes)
		
		else:
			data = data.astype("float32")
			data /= 255 #convert the value to 0 ~ 1 scale
			shape = (data.shape[0],) + self.image_shape
			data = data.reshape(shape)
			
		return data
	
	def get_batch(self):
		# x: data, y: lebel
		(x_train, y_train), (x_test, y_test) = cifar10.load_data()
		
		x_train, x_test = [self.preprocess(d) for d in [x_train, x_test]]
		y_train, y_test = [self.preprocess(d, label_data=True) for d in
					 [y_train, y_test]]
		
		return x_train, y_train, x_test, y_test

## TensorBoard用のログファイル生成関数

In [0]:
from __future__ import absolute_import
from __future__ import unicode_literals
from time import gmtime, strftime
from keras.callbacks import TensorBoard
import os


def make_tensorboard(set_dir_name=''):
    tictoc = strftime("%a_%d_%b_%Y_%H_%M_%S", gmtime())
    directory_name = tictoc
    log_dir = set_dir_name + '_' + directory_name
    os.mkdir(log_dir)
    tensorboard = TensorBoard(log_dir=log_dir, write_graph=True, )
    return tensorboard

## Training, Evaluation用クラスの定義

In [0]:
import os
from keras.models import load_model
from keras.callbacks import TensorBoard, ModelCheckpoint, LearningRateScheduler
from keras.optimizers import RMSprop, Adam, Nadam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
import numpy as np

from __future__ import absolute_import
from __future__ import unicode_literals
from time import gmtime, strftime
import os


def make_tensorboard(set_dir_name=''):
    tictoc = strftime("%a_%d_%b_%Y_%H_%M_%S", gmtime())
    directory_name = tictoc
    log_dir = set_dir_name + '_' + directory_name
    os.mkdir(log_dir)
    tensorboard = TensorBoard(log_dir=log_dir, write_graph=True, )
    return tensorboard

class Trainer():
	
	def __init__(self, model, loss, optimizer, logdir = './'):
		self._target = model
		self._target.compile(
				loss=loss, optimizer=optimizer, metrics=["accuracy"]
				)
		self.verbose = 1 # visualize progress bar: 0(OFF), 1(On), 2(On:each data) 
		#self.log_dir = os.path.join(os.path.dirname(__file__), logdir)
		self.log_dir = os.path.join(logdir)
		self.model_file_name = "model_file.hdf5"
	
	def train_for_tuning_test_data(self, 
            x_train, y_train, x_test, y_test, batch_size, epochs, lr_scheduler):
		datagen = ImageDataGenerator(
			featurewise_center=False,            # set input mean to 0 over the dataset
            samplewise_center=False,             # set each sample mean to 0
            featurewise_std_normalization=False, # divide inputs by std
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,                 # apply ZCA whitening
            rotation_range=20,                   # randomly rotate images in the range (0~180)
            width_shift_range=0.2,               # randomly shift images horizontally
            height_shift_range=0.2,              # randomly shift images vertically
            zoom_range = 0.2,
            channel_shift_range = 0.2,
            horizontal_flip=True,                # randomly flip images
            vertical_flip=False                  # randomly flip images
		)
        
    # training (validation dataはデータ拡張はしない)
		model_path = os.path.join(self.log_dir, self.model_file_name)
		self._target.fit_generator(
            generator        = datagen.flow(x_train,y_train, batch_size),
            steps_per_epoch  = x_train.shape[0] // batch_size,
            epochs           = epochs,
            validation_data  = ImageDataGenerator().flow(x_test,y_test, batch_size),
			      validation_steps = x_test.shape[0] // batch_size,
            callbacks=[
                LearningRateScheduler(lr_scheduler),
                make_tensorboard(set_dir_name=self.log_dir),
                ModelCheckpoint(model_path, save_best_only=True)
            ],
            verbose = self.verbose,
            workers = 4
        )
		

class Evaluator():
    
    def __init__(self, result_file_path="./prediction_result.csv"):
        self.result_file_path="./prediction_result.csv"
        
    def simple_evaluate(self, model, x_test, label):
        print("start evaluation...")
        score = model.evaluate(x_test, y_test, verbose=1)
        print("Test loss:", score[0])
        print("Test accuracy:", score[1])
        return score
    
    def tta_evaluate(self, model, x_test, label, batch_size = 2500, tta_epochs = 2):
        print("batch size (TTA): "+str(batch_size))
        print("epochs (TTA): "+str(tta_epochs))
        tta = TTA()
        tta_pred = tta.predict(model, x_test, batch_size, epochs = tta_epochs)
        print("Test accuracy(TTA): ",end = "")
        print( accuracy_score( np.argmax(tta_pred,axis = 1) , np.argmax(label,axis = 1)))
        return tta_pred

## 学習率減衰

In [0]:
def learning_rate_schedule_for_Adam(epoch):
	lr = 0.001
	if(epoch >= 100): lr = 0.0002 
	if(epoch >= 140): lr = 0.0001
	return lr

## 実行

In [9]:
from keras.optimizers import Adam



# create dataset
dataset = CIFAR10Dataset()
x_train, y_train, x_test, y_test = dataset.get_batch()

# create model
model = deep_cnn(dataset.image_shape, dataset.num_classes)

# train the model
# RMSprpの方がいいかもしれない
trainer = Trainer(model, loss="categorical_crossentropy", optimizer=Adam(), logdir='/content/drive/My Drive/Colab Notebooks/Logs/deep_cnn_2')
trainer.train_for_tuning_test_data(
            x_train, y_train, x_test, y_test, batch_size=500, epochs=150, 
            lr_scheduler=learning_rate_schedule_for_Adam)
	

# show result
evaluator = Evaluator()
score = evaluator.simple_evaluate(model, x_test, y_test)

170500096/170498071 [==============================] - 13s 0us/step


W0818 05:50:44.914837 139961580173184 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0818 05:50:44.951925 139961580173184 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0818 05:50:44.962502 139961580173184 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0818 05:50:45.018525 139961580173184 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0818 05:50:45.019335 1399615801

Epoch 1/150
100/100 [==============================] - 43s 430ms/step - loss: 1.9527 - acc: 0.2352 - val_loss: 2.3619 - val_acc: 0.2866
Epoch 2/150
100/100 [==============================] - 32s 322ms/step - loss: 1.6230 - acc: 0.3847 - val_loss: 3.4643 - val_acc: 0.2909
Epoch 3/150
100/100 [==============================] - 33s 329ms/step - loss: 1.4136 - acc: 0.4789 - val_loss: 3.1220 - val_acc: 0.2869
Epoch 4/150
100/100 [==============================] - 33s 329ms/step - loss: 1.2674 - acc: 0.5440 - val_loss: 2.6944 - val_acc: 0.3800
Epoch 5/150
100/100 [==============================] - 33s 329ms/step - loss: 1.1484 - acc: 0.5946 - val_loss: 1.4794 - val_acc: 0.5477
Epoch 6/150
100/100 [==============================] - 33s 331ms/step - loss: 1.0474 - acc: 0.6317 - val_loss: 1.8338 - val_acc: 0.5064
Epoch 7/150
100/100 [==============================] - 33s 332ms/step - loss: 0.9897 - acc: 0.6556 - val_loss: 1.1791 - val_acc: 0.6309
Epoch 8/150
100/100 [===========================

##  Test Time Augmentation（TTA）を用いた推論 

In [20]:
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

class TTA:
    
    #test_time_augmentation
    #batch_sizeは，test_sizeの約数!!!
    def predict(self, model, x_test, batch_size ,epochs = 10):
        
        # Augmentation用generatorによるデータセットの作成
        data_flow = self.generator(x_test, batch_size)
        
        test_size = x_test.shape[0]
        pred = np.zeros(shape = (test_size,10), dtype = float)
        
        step_per_epoch = test_size //batch_size
        for epoch in range(epochs):
            print( 'epoch: ' + str(epoch+1)+'/'+str(epochs))
            for step in range(step_per_epoch):
                #print( 'step: ' + str(step+1)+'/'+str(step_per_epoch))
                sta = batch_size * step
                end = sta + batch_size
                tmp_x = data_flow.__next__()
                pred[sta:end] += model.predict(tmp_x)        
        return pred / epochs
    
    
    def generator(self, x_test,batch_size):
        return ImageDataGenerator(
                    rotation_range = 20,
                    horizontal_flip = True,
                    height_shift_range = 0.2,
                    width_shift_range = 0.2,
                    zoom_range = 0.2,
                    channel_shift_range = 0.2
                ).flow(x_test,batch_size = batch_size,shuffle = False)

      
# show result
evaluator = Evaluator()
score = evaluator.tta_evaluate(model, x_test, y_test, batch_size = 500, tta_epochs = 50)

batch size (TTA): 500
epochs (TTA): 50
epoch: 1/50
epoch: 2/50
epoch: 3/50
epoch: 4/50
epoch: 5/50
epoch: 6/50
epoch: 7/50
epoch: 8/50
epoch: 9/50
epoch: 10/50
epoch: 11/50
epoch: 12/50
epoch: 13/50
epoch: 14/50
epoch: 15/50
epoch: 16/50
epoch: 17/50
epoch: 18/50
epoch: 19/50
epoch: 20/50
epoch: 21/50
epoch: 22/50
epoch: 23/50
epoch: 24/50
epoch: 25/50
epoch: 26/50
epoch: 27/50
epoch: 28/50
epoch: 29/50
epoch: 30/50
epoch: 31/50
epoch: 32/50
epoch: 33/50
epoch: 34/50
epoch: 35/50
epoch: 36/50
epoch: 37/50
epoch: 38/50
epoch: 39/50
epoch: 40/50
epoch: 41/50
epoch: 42/50
epoch: 43/50
epoch: 44/50
epoch: 45/50
epoch: 46/50
epoch: 47/50
epoch: 48/50
epoch: 49/50
epoch: 50/50
Test accuracy(TTA): 0.9412
